<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Create Issue from Naas Chat
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Create+Issue+from+Naas+Chat:+Error+short+description">Bug report</a>

**Tags:** #github #naas #naas_driver #command #chat

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-09-28 (Created: 2023-09-28)

**Description:** This notebook creates an issue on a given GitHub repository from Naas Chat.

**References:**
- [Generate a personal access token](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token)
- [Go to personal access token](https://github.com/settings/tokens)

## Input

### Import libraries

In [ ]:
import naas
import requests
import json

### Setup variables
**Mandatory**
- `command_name`: The name of the command.
- `message`: Message to be returned
- `repo_url`: GitHub repo HTML url
- `title`: Issue title
- `description`: Issue description
- `assignee`: Issue assignee.
- `label`: Issue label

**Optional**
- `body`: This variable stores the body to be send by the webhook.
- `model`: The name of the model to be used for tokenization. Default is "gpt-3.5-turbo-16k".
- `temperature`: The temperature parameter for the model. Default is 0.
- `output_path`: The path where the JSON file should be saved. If not provided, it will be created from the plugin name.

In [ ]:
# Mandatory
command_name = "Create_Issue"
message = f"✅ Issue has been created, here is the link: [URL]"
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
token = naas.secret.get('GITHUB_TOKEN')
title = ""
description = ""
assignee = ""
label = ""

# Optional
body = {}
model = "gpt-3.5-turbo-16k"
temperature = 0
output_path = None

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    repo_url = body.get("repo_url") if body.get("repo_url") else "https://github.com/jupyter-naas/awesome-notebooks"
    title = body.get("title")
    description = body.get("description")
    assignee = body.get("assignee")
    label = body.get("label")

## Model

### Create New GitHub Issue

In [ ]:
def create_new_github_issue(
    repo_url,
    token,
    title,
    description,
    assignee,
    label,
):  
    # Init
    status = "ok"   
    try:
        # Repo name
        repo_name = repo_url.split("https://github.com/")[-1]

        # Requests
        url = f"https://api.github.com/repos/{repo_name}/issues"
        data = {}
        if title:
            data["title"] = title
        if description:
            data["body"] = description
        if assignee:
            data["assignees"] = assignee
        if label:
            data["labels"] = label
        if len(data) > 0:
            headers = {'Authorization': f'token {token}'}
            response = requests.post(url, data=json.dumps(data), headers=headers)
            github_issue = response.json()
            url = f"https://github.com/jupyter-naas/awesome-notebooks/issues/{github_issue.get('number')}"
            message = message.replace("[URL]", url)
        else:
            message = f"Nothing to create, payload is empty: {data}."
    except Exception as e:
        status = "ko"
        message = e
        print(e)
    return status, message

status, message = create_new_github_issue(
    repo_url,
    token,
    title,
    description,
    assignee,
    label,
)
print("Status:", status)
print("Message:", message)

## Output

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)